In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('mimic_HF.csv')

In [3]:
duplicate_column = 'subject_id'
df.drop_duplicates(subset=duplicate_column, inplace=True)

In [4]:
columns_to_drop = ['subject_id_1', 'subject_id_2', 'SUBJECT_ID_3', 'min_row_id', 'ROW_ID', 'CHARTDATE', 'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'subject_id_4', 'subject_id_5', 'age_1', 'TEXT', 'HADM_ID_1', 'hadm_id', ]
df = df.drop(columns=columns_to_drop)

In [5]:
df

,subject_id,icd9_hypertension,hypertension_admit_age,hypertension_icu_stay,icd9_myocardialinfarction,myocardialinfarction_admit_age,myocardialinfarction_icu_stay,icd9_diabetesmellitus,diabetesmellitus_admit_age,diabetesmellitus_icu_stay,...,HF_admit_age,height,weight,bpsys,bpdias,hr,age,echo_icu_stay,gender,V1
0,23224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.97,67.0,125.0,120.0,80.0,86.0,54.97,1,F,55.0
1,30726,NaN,NaN,NaN,NaN,NaN,NaN,25000.0,79.54,1.0,...,NaN,66.0,150.0,134.0,61.0,60.0,79.54,1,F,35.0
2,71190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,68.0,220.0,149.0,90.0,60.0,54.78,1,M,55.0
3,24499,4019.0,52.37,1.0,41071.0,52.37,1.0,25001.0,52.37,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,52.40,1,M,55.0
6,88982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,87.30,71.0,90.0,130.0,70.0,40.0,87.30,1,M,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39173,31902,4019.0,66.97,1.0,NaN,NaN,NaN,25082.0,66.97,1.0,...,66.97,66.0,182.0,149.0,86.0,76.0,66.97,1,F,20.0
39175,5118,4019.0,53.39,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,53.39,64.0,136.0,126.0,67.0,75.0,53.39,1,F,55.0
39176,23440,4019.0,80.73,1.0,41071.0,80.73,1.0,25000.0,80.45,1.0,...,80.55,61.0,144.0,160.0,65.0,60.0,80.55,1,F,50.0
39181,24489,NaN,NaN,NaN,NaN,NaN,NaN,25070.0,65.71,1.0,...,NaN,64.0,191.0,100.0,40.0,NaN,65.72,1,F,55.0


In [6]:
column_name = 'target_HF'
nan_mask = df[column_name].isna()
num_nan = nan_mask.sum()
print(num_nan)
print(f'The number of positive values in {column_name} column is: {len(df) - num_nan}')

12479
The number of positive values in target_HF column is: 7917


In [7]:
#Count the number of diagnosis risk factors
icd9_cols = df.filter(regex='^icd9').columns
num_icd9_cols = icd9_cols.shape[0]

print("Number of columns that start with 'icd9':", num_icd9_cols)
icd9_cols

Number of columns that start with 'icd9': 39


Index(['icd9_hypertension', 'icd9_myocardialinfarction',
       'icd9_diabetesmellitus', 'icd9_renalfailure',
       'icd9_aorticvalvedisorders', 'icd9_amyloidosis', 'icd9_cancer',
       'icd9_thyroiddisorder', 'icd9_conduction', 'icd9_irondeficiency',
       'icd9_sepsis', 'icd9_connectivetissuedisease',
       'icd9_rheumatoidarthritis', 'icd9_tricuspidvalvedisorders',
       'icd9_thyrotoxicosis', 'icd9_mitralvalvedisorder', 'icd9_radiation',
       'icd9_parathyroid', 'icd9_metabolicsyndrome',
       'icd9_pregnancypreeclampsia', 'icd9_rickettsialpox', 'icd9_sarcoidosis',
       'icd9_aneurysmcoronaryartery', 'icd9_HIV',
       'icd9_pulmonaryvalvedisorder', 'icd9_malnutrition',
       'icd9_hypertrophiccardiomyopathy', 'icd9_arteriovenousfistula',
       'icd9_lupuserythematosus', 'icd9_anorexianervosa', 'icd9_cocaine',
       'icd9_musculardystrophies', 'icd9_acromegaly', 'icd9_conns',
       'icd9_endocardialfibroelastosis', 'icd9_leadtoxicity',
       'icd9_thiaminedeficiency'

In [8]:
hf_age = df['HF_admit_age']

# Find all columns with age of diagnoses
age_cols = df.filter(regex='admit_age').columns
age_cols = age_cols[:-1] #don't get the HF age cause we don't want to drop that one

# Find all columns with icu stay of diagnoses
icu_cols = df.filter(regex='icu_stay').columns

# Find columns associated with echo data
echo_cols = ['height', 'weight', 'bpsys', 'bpdias', 'hr']
icu_cols

Index(['hypertension_icu_stay', 'myocardialinfarction_icu_stay',
       'diabetesmellitus_icu_stay', 'renalfailure_icu_stay',
       'aorticvalvedisorder_icu_stay', 'amyloidosis_icu_stay',
       'cancer_icu_stay', 'thyroiddisorder_icu_stay', 'conduction_icu_stay',
       'irondeficiency_icu_stay', 'sepsis_icu_stay',
       'connectivetissuedisease_icu_stay', 'rheumatoidarthritis_icu_stay',
       'tricuspidvalvedisorders_icu_stay', 'thyrotoxicosis_icu_stay',
       'mitralvalvedisorder_icu_stay', 'radiation_icu_stay',
       'parathyroid_icu_stay', 'metabolicsyndrome_icu_stay',
       'pregnancypreeclampsia_icu_stay', 'rickettsialpox_icu_stay',
       'sarciodosis_icu_stay', 'aneurysmcoronaryartery_icu_stay',
       'HIV_icu_stay', 'pulmonaryvalvedisease_icu_stay',
       'malnutrition_icu_stay', 'hypertrophiccardiomyopathy_icu_stay',
       'arteriovenousfistula_icu_stay', 'lupuserythematosus_icu_stay',
       'anorexianervosa_icu_stay', 'cocaine_icu_stay',
       'musculardystrophie

In [9]:
#Check if age at diagnosis is less than age at HF diagnosis
for index, row in df.iterrows():
    if pd.isna(row['HF_admit_age']):
        #check the echo age ?
        pass
    else:
        # don't use a diagnosis if it happened after the HF diagnosis
        for diagnosis_age in age_cols:
            if row[diagnosis_age] > row['HF_admit_age']:
                column_index = df.columns.get_loc(diagnosis_age)
                
                #set all values related to the diagnosis to 0 if it happened after the heart failure diagnosis
                row.iloc[column_index - 1] = np.nan
                row.iloc[column_index] = np.nan
                row.iloc[column_index + 1] = np.nan
        
        # don't use data from the echo if it happened after HF diagnosis
        if row['age'] > row['HF_admit_age']:
            for col in echo_cols:
                row[col] = np.nan
        

In [10]:
df

,subject_id,icd9_hypertension,hypertension_admit_age,hypertension_icu_stay,icd9_myocardialinfarction,myocardialinfarction_admit_age,myocardialinfarction_icu_stay,icd9_diabetesmellitus,diabetesmellitus_admit_age,diabetesmellitus_icu_stay,...,HF_admit_age,height,weight,bpsys,bpdias,hr,age,echo_icu_stay,gender,V1
0,23224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.97,67.0,125.0,120.0,80.0,86.0,54.97,1,F,55.0
1,30726,NaN,NaN,NaN,NaN,NaN,NaN,25000.0,79.54,1.0,...,NaN,66.0,150.0,134.0,61.0,60.0,79.54,1,F,35.0
2,71190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,68.0,220.0,149.0,90.0,60.0,54.78,1,M,55.0
3,24499,4019.0,52.37,1.0,41071.0,52.37,1.0,25001.0,52.37,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,52.40,1,M,55.0
6,88982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,87.30,71.0,90.0,130.0,70.0,40.0,87.30,1,M,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39173,31902,4019.0,66.97,1.0,NaN,NaN,NaN,25082.0,66.97,1.0,...,66.97,66.0,182.0,149.0,86.0,76.0,66.97,1,F,20.0
39175,5118,4019.0,53.39,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,53.39,64.0,136.0,126.0,67.0,75.0,53.39,1,F,55.0
39176,23440,4019.0,80.73,1.0,41071.0,80.73,1.0,25000.0,80.45,1.0,...,80.55,61.0,144.0,160.0,65.0,60.0,80.55,1,F,50.0
39181,24489,NaN,NaN,NaN,NaN,NaN,NaN,25070.0,65.71,1.0,...,NaN,64.0,191.0,100.0,40.0,NaN,65.72,1,F,55.0


In [11]:
for col in icd9_cols:
    df[col] = df[col].mask(df[col].notnull(), 1)
    df[col].fillna(0, inplace=True)
    
for col in age_cols:
    df[col].fillna(-1, inplace=True)
    
for col in icu_cols:
    df[col].fillna(-1, inplace=True)
    
for col in echo_cols:
    df[col].fillna(-1, inplace=True)
    
#Rename EF and clean that column
df.rename(columns={'V1': 'EF'}, inplace=True)
df['EF'].fillna(-1, inplace=True)

# Clean HF data columns
df['target_HF'] = df['target_HF'].mask(df['target_HF'].notnull(), 1)
df['target_HF'].fillna(0, inplace=True)
df['HF_admit_age'].fillna(-1, inplace=True)

df

,subject_id,icd9_hypertension,hypertension_admit_age,hypertension_icu_stay,icd9_myocardialinfarction,myocardialinfarction_admit_age,myocardialinfarction_icu_stay,icd9_diabetesmellitus,diabetesmellitus_admit_age,diabetesmellitus_icu_stay,...,HF_admit_age,height,weight,bpsys,bpdias,hr,age,echo_icu_stay,gender,EF
0,23224,0.0,-1.00,-1.0,0.0,-1.00,-1.0,0.0,-1.00,-1.0,...,54.97,67.0,125.0,120.0,80.0,86.0,54.97,1,F,55.0
1,30726,0.0,-1.00,-1.0,0.0,-1.00,-1.0,1.0,79.54,1.0,...,-1.00,66.0,150.0,134.0,61.0,60.0,79.54,1,F,35.0
2,71190,0.0,-1.00,-1.0,0.0,-1.00,-1.0,0.0,-1.00,-1.0,...,-1.00,68.0,220.0,149.0,90.0,60.0,54.78,1,M,55.0
3,24499,1.0,52.37,1.0,1.0,52.37,1.0,1.0,52.37,1.0,...,-1.00,-1.0,-1.0,-1.0,-1.0,-1.0,52.40,1,M,55.0
6,88982,0.0,-1.00,-1.0,0.0,-1.00,-1.0,0.0,-1.00,-1.0,...,87.30,71.0,90.0,130.0,70.0,40.0,87.30,1,M,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39173,31902,1.0,66.97,1.0,0.0,-1.00,-1.0,1.0,66.97,1.0,...,66.97,66.0,182.0,149.0,86.0,76.0,66.97,1,F,20.0
39175,5118,1.0,53.39,1.0,0.0,-1.00,-1.0,0.0,-1.00,-1.0,...,53.39,64.0,136.0,126.0,67.0,75.0,53.39,1,F,55.0
39176,23440,1.0,80.73,1.0,1.0,80.73,1.0,1.0,80.45,1.0,...,80.55,61.0,144.0,160.0,65.0,60.0,80.55,1,F,50.0
39181,24489,0.0,-1.00,-1.0,0.0,-1.00,-1.0,1.0,65.71,1.0,...,-1.00,64.0,191.0,100.0,40.0,-1.0,65.72,1,F,55.0
